# DWPose Inference in Amazon SageMaker

This notebook will demonstrate how to create an endpoint for real time inference with the pretrained DWPose.

## 1. SageMaker Initialization 
First we upgrade SageMaker to the latest version. If your notebook is already using latest Sagemaker 2.x API, you may skip the next cell.

In [1]:
! pip install --upgrade pip
! python3 -m pip install --upgrade sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
  Using cached sagemaker-2.226.1-py3-none-any.whl.metadata (15 kB)
Using cached sagemaker-2.226.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.224.4
    Uninstalling sagemaker-2.224.4:
      Successfully uninstalled sagemaker-2.224.4


In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

role = (
    get_execution_role()
)  # provide a pre-existing role ARN as an alternative to creating a new role

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

session = boto3.session.Session()
sess = sagemaker.session.Session()
aws_region = session.region_name

sagemaker_session = sagemaker.session.Session(boto_session=session)

container_name = "dwpose-serving"

## 2. Build and Push Amazon SageMaker Serving Container Images

For this step, the [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) attached to this notebook instance needs full access to [Amazon ECR service](https://aws.amazon.com/ecr/).

### 2.1 Docker Environment Preparation

Because the volume size of container may be larger than the available size in root directory of the notebook instance, we need to put the directory of docker data into the ```/home/ec2-user/SageMaker/docker``` directory.

By default, the root directory of docker is set as ```/var/lib/docker/```. We need to change the directory of docker to ```/home/ec2-user/SageMaker/docker```.

In [3]:
!bash ./prepare-docker.sh

Redirecting to /bin/systemctl stop docker.service
  docker.socket
Redirecting to /bin/systemctl start docker.service


### 2.2 Build and Push DWPose Serving Container Image

Use [`./container-serving/build_tools/build_and_push.sh`](./container-serving/build_tools/build_and_push.sh) script to build the [DWPose](https://github.com/IDEA-Research/DWPose) <b>serving</b> container image and push it to Amazon ECR. 

In [4]:
!cat ./{container_name}/build_tools/build_and_push.sh

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.

DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source $DIR/set_env.sh

# set region
region=
if [ "$#" -eq 1 ]; then
    region=$1
else
    echo "usage: $0 <aws-region>"
    exit 1
fi

image=$IMAGE_NAME
tag=$IMAGE_TAG

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:${tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --region ${region} --repository-names "${image}" > /dev/null 2>&1
if [ $? -ne 0 ]; then
    aws ecr create-repository --region ${reg

Using your *AWS region* as argument, run the cell below.

In [ ]:
%%time
! ./{container_name}/build_tools/build_and_push.sh {aws_region}

In [6]:
serving_image = f"{account}.dkr.ecr.{aws_region}.amazonaws.com/dwpose:pytorch-serving"

## 3. Create Inference Endpoint

### 3.1 Define Amazon SageMaker Model
Next, we define an Amazon SageMaker Model that we will serve from an Amazon SageMaker Endpoint. 

In [7]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
model_name = f"dwpose-{timestamp}" # set the name of the model

You can get the S3 URI of the trained model in Training job console once the training job gets finished, and then set `ModelDataUrl` to the S3 URI of the trained model.

In [ ]:
model_bucket = sagemaker_session.default_bucket()

prefix = "video-generation/dwpose"
s3_model_uri = f"s3://{model_bucket}/{prefix}/pretrained-model/model.tar.gz"

!cd ./pretrained_weights/DWPose && tar -czvf model.tar.gz .
!aws s3 cp ./pretrained_weights/DWPose/model.tar.gz $s3_model_uri
!rm -r ./pretrained_weights/DWPose/model.tar.gz

In [9]:
# Restore the s3 uri of trained model
#s3_model_uri="s3://sagemaker-us-east-1-822507008821/video-generation/dwpose/pretrained-model/model.tar.gz" # use the s3 uri of the finetuned or pretrained model

serving_container_def = {
    'Image': serving_image,
    'ModelDataUrl': s3_model_uri,
    'Mode': 'SingleModel',
    'Environment': {
                    'SM_MODEL_DIR' : '/opt/ml/model',
                   }
}

create_model_response = sagemaker_session.create_model(name=model_name, 
                                                       role=role, 
                                                       container_defs=serving_container_def)

 ### 3.2 Create Endpoint Configuration
 Next, we set the name of the Amaozn SageMaker hosted service endpoint configuration.

In [10]:
endpoint_config_name = f"{model_name}-endpoint-config"
print(endpoint_config_name)

dwpose-20240723-073243-endpoint-config


Then create the Amazon SageMaker hosted service endpoint configuration that uses one instance of `ml.g5.xlarge` to serve the model.

In [11]:
epc = sagemaker_session.create_endpoint_config(
    name=endpoint_config_name,
    model_name=model_name,
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",
)
print(epc)

dwpose-20240723-073243-endpoint-config


Next we specify the Amazon SageMaker endpoint name for the endpoint used to serve the model.

In [12]:
endpoint_name = f"{model_name}-endpoint"
print(endpoint_name)

dwpose-20240723-073243-endpoint


### 3.3 Create Endpoint
In this step, we create the Amazon SageMaker endpoint using the endpoint configuration we created above.

In [ ]:
ep = sagemaker_session.create_endpoint(
    endpoint_name=endpoint_name, config_name=endpoint_config_name, wait=True
)
print(ep)

---------------

## 4. Test Endpoint

### 4.2 Invoke endpoint

In [15]:
%%bash
pip install opencv-python
pip install av
pip install torch
pip install torchvision
pip install einops

In [16]:
%%writefile dwpose_inference.py
import boto3
import base64
import json
import cv2
import time
import os
from io import BytesIO
from PIL import Image
import numpy as np
import sys
from tqdm import tqdm

sys.path.append("./Moore-AnimateAnyone/src")
from utils.util import save_videos_from_pil 

client = boto3.client("sagemaker-runtime")

def extract_dwpose(data_path, endpoint_name, file_path):
    cap = cv2.VideoCapture(data_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    #processing_time = 0
    frame_id = 0
    pose_frames = []

    with tqdm(total=total_frames, desc="Processing frames") as pbar:
        while True:
            ret, frame = cap.read()
            if ret != True:
                break
            data = {}

            _, buffer = cv2.imencode('.jpg', frame)
            img_data = base64.b64encode(buffer)
            data["images"] = [img_data.decode("utf-8")]
            body = json.dumps(data).encode("utf-8")

            request_time=time.time()
            response = client.invoke_endpoint(
                EndpointName=endpoint_name, ContentType="application/json", Accept="application/json", Body=body
            )
            #if frame_id > 0:
            #    processing_time += (time.time() - request_time)
            body = response["Body"].read()
            results = body.decode("utf-8")
            pose_list = json.loads(results)
            pose_images = []
            for pose in pose_list:
                with BytesIO(base64.b64decode(pose)) as f:
                    pose_frames.append(Image.open(f).copy()) 
            frame_id += 1
            # Update the progress bar
            pbar.update(1)
            pbar.set_postfix({'Frame': frame_id, 'Processing time': f'{time.time() - request_time:.2f}s'})

    save_videos_from_pil(pose_frames, file_path, fps=fps)
    cap.release()

Overwriting dwpose_inference.py


In [18]:
from dwpose_inference import extract_dwpose
import os

data_path = "./Moore-AnimateAnyone/configs/inference/talkinghead_videos/1.mp4"
save_dir = "datasets/"

#endpoint_name = "dwpose-20240723-073243-endpoint"

file_name = os.path.basename(data_path)
file_dir = os.path.dirname(data_path)
file_name = file_name if file_dir != save_dir else f'gen_{file_name}'
file_path = os.path.join(save_dir, file_name)

extract_dwpose(data_path, endpoint_name, file_path)

Processing frames: 100%|██████████| 299/299 [00:31<00:00,  9.54it/s, Frame=299, Processing time=0.05s]


In [19]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/jupyterlab/debuglog.py:54: UserWarning: An error occurred.
  warnings.warn("An error occurred.")
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/jupyterlab/debuglog.py:55: UserWarning: ValueError: Please install Node.js and npm before continuing installation. You may be able to install Node.js from your package manager, from conda, or directly from the Node.js website (https://nodejs.org).
  warnings.warn(msg[-1].strip())
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/jupyterlab/debuglog.py:56: UserWarning: See the log file for details: /tmp/jupyterlab-de

In [20]:
from IPython.display import HTML

video_html = f"""
<video width="640" height="480" controls>
  <source src="{file_path}" type="video/mp4">
  Your browser does not support the video tag.
</video>
"""
HTML(video_html)

The response from the endpoint includes the bounding box information and ID for each person. You can download the processed video from `datasets` to check from the local instance.

## Delete SageMaker Endpoint, Endpoint Config and Model
<span style="color:red;">**If you are done testing, delete the deployed Amazon SageMaker endpoint, endpoint config, and the model below.**</span>

In [21]:
sagemaker_session.delete_endpoint(endpoint_name=endpoint_name)
sagemaker_session.delete_endpoint_config(endpoint_config_name=endpoint_config_name)
sagemaker_session.delete_model(model_name=model_name)